### Reranking

In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain_core.output_parsers import StrOutputParser 

In [2]:
# load the text file
loader = TextLoader("langchain_sample.txt")
raw_docs = loader.load()
raw_docs

[Document(metadata={'source': 'langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.\nLangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.\nRetrieval-Augmented Generation (RAG) is a powerful technique where external knowledge is retrieved and passed into the prompt to ground LLM responses. LangChain makes it easy to implement RAG using vector databases like FAISS, Chroma, and Pinecone.\nBM25 is a traditional sparse retrieval method that scores documents based on keyword matching. Although fast, it often struggles with synonyms and semantic similari

In [4]:
# Split the docs in to chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300, 
    chunk_overlap = 50
)

docs = splitter.split_documents(raw_docs)
len(docs)

10

In [6]:
## User query 
query =  "How can i use langchain to use app with memory and tools"
query

'How can i use langchain to use app with memory and tools'

In [9]:
### FAISS and Huggingface model Embeddings

from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

embedding_model=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore=FAISS.from_documents(docs,embedding_model)
retriever=vectorstore.as_retriever(search_kwargs={"k":8})

In [10]:
## OpenAI Embedding
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
from langchain_openai import OpenAIEmbeddings

embeddings=OpenAIEmbeddings()
vectorstore_openai=FAISS.from_documents(docs,embeddings)
retriever_openai=vectorstore_openai.as_retriever(search_kwargs={"k":8})




In [11]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000245C12856D0>, search_kwargs={'k': 8})

In [12]:
retriever_openai

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000245E6BF2C10>, search_kwargs={'k': 8})

In [29]:
# Model

from langchain_openai import ChatOpenAI
llm  = ChatOpenAI(model = "gpt-5-nano")

In [30]:
# Prompt Template
prompt = PromptTemplate.from_template("""
You are a helpful assistant. Your task is to rank the following documents from most to least relevant to the user's question.

User Question: "{question}"

Documents:
{documents}

Instructions:
- Think about the relevance of each document to the user's question.
- Return a list of document indices in ranked order, starting from the most relevant.

Output format: comma-separated document indices (e.g., 2,1,3,0,...)
""")

In [31]:
retrieved_docs=retriever.invoke(query)
retrieved_docs

[Document(id='de206e83-c01c-49df-9f15-539278844a96', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.'),
 Document(id='13dcf382-0bee-4e71-bb48-43756fc7c7e7', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.'),
 Document(id='f668ce20-81a1-4448-be8b-d644799bd69b', metadata={'source': 'langchain_sample.txt'}, page_content='Memory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and statefu

In [32]:
chain=prompt| llm | StrOutputParser()
chain

PromptTemplate(input_variables=['documents', 'question'], input_types={}, partial_variables={}, template='\nYou are a helpful assistant. Your task is to rank the following documents from most to least relevant to the user\'s question.\n\nUser Question: "{question}"\n\nDocuments:\n{documents}\n\nInstructions:\n- Think about the relevance of each document to the user\'s question.\n- Return a list of document indices in ranked order, starting from the most relevant.\n\nOutput format: comma-separated document indices (e.g., 2,1,3,0,...)\n')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000245EB48FD90>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000245EB48F890>, root_client=<openai.OpenAI object at 0x00000246656609D0>, root_async_client=<openai.AsyncOpenAI object at 0x00000246656603E0>, model_name='gpt-5-nano', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [33]:
doc_lines = [f"{i+1}. {doc.page_content}" for i, doc in enumerate(retrieved_docs)]
formatted_docs = "\n".join(doc_lines)

In [34]:
doc_lines

['1. LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.',
 '2. LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.',
 '3. Memory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.',
 '4. LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.',
 '5. Agents in LangChain are chains that use LLMs to decide which tools to use and in what order. This makes them suitable for multi-s

In [35]:
formatted_docs

'1. LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.\n2. LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models and optimize performance for specific use cases like summarization, question answering, or translation.\n3. Memory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.\n4. LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.\n5. Agents in LangChain are chains that use LLMs to decide which tools to use and in what order. This makes them suitable for multi-step tasks lik

In [36]:
response=chain.invoke({"question":query,"documents":formatted_docs})
response

'3,4,5,1,2,7,6,8'

In [37]:
# Step 5: Parse and rerank
indices = [int(x.strip()) - 1 for x in response.split(",") if x.strip().isdigit()]
indices

[2, 3, 4, 0, 1, 6, 5, 7]

In [38]:
reranked_docs = [retrieved_docs[i] for i in indices if 0 <= i < len(retrieved_docs)]
reranked_docs

[Document(id='f668ce20-81a1-4448-be8b-d644799bd69b', metadata={'source': 'langchain_sample.txt'}, page_content='Memory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.'),
 Document(id='70277a8f-9e1c-4c0c-84a3-9234c9d55db3', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.'),
 Document(id='0dac0209-b3d5-4567-9ac5-4bae00efef4d', metadata={'source': 'langchain_sample.txt'}, page_content='Agents in LangChain are chains that use LLMs to decide which tools to use and in what order. This makes them suitable for multi-step tasks like question answering with search and code execution.'),
 Document(id='de206e83-c01c-49df-9f15-539278844a96', metadata={'source': 'langchain_sample.txt'}, page_content='LangChain is a flexible f

In [39]:
# Step 6: Show results
print("\n📊 Final Reranked Results:")
for i, doc in enumerate(reranked_docs, 1):
    print(f"\nRank {i}:\n{doc.page_content}")


📊 Final Reranked Results:

Rank 1:
Memory in LangChain enables context retention across multiple steps in a conversation or task, making the application more coherent and stateful.

Rank 2:
LangChain supports tool integration including web search, calculators, and APIs, allowing LLMs to interact with external systems and respond more accurately to dynamic queries.

Rank 3:
Agents in LangChain are chains that use LLMs to decide which tools to use and in what order. This makes them suitable for multi-step tasks like question answering with search and code execution.

Rank 4:
LangChain is a flexible framework designed for developing applications powered by large language models (LLMs). It provides tools and abstractions to work with LLMs more effectively and includes components for prompt management, chains, memory, and agents.

Rank 5:
LangChain integrates with many third-party services such as OpenAI, Hugging Face, and Cohere. This enables developers to experiment with different models